In [1]:
#  Import libraries and define variables
import matplotlib.pyplot as plt
import os
import shutil
import glob
from tqdm import tqdm
from PIL import Image
import nibabel as nib
import numpy as np
import cv2
import pickle
import cv2
import tensorflow as tf
import matplotlib
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics
matplotlib.use('TkAgg')


# Define the modalities and classifications
modalities = ['t1', 't1ce', 't2', 'flair']
classifications = ['mgmt_positive', 'mgmt_negative']

# Define patch size and stride
block_h, block_w = 32, 32
stride = 2

# Interpolated image dimestions
inter_dim = (110, 90)

# Loading model
load_model = 'latest_model.h5'

# Define paths to the BraTS dataset folders
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

PATH = path + 'Data/BRATS/novel_data/'
Test_Dir = PATH + 'Working_Data/test/'
Test_Dir1 = PATH + 'test/'
Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = PATH + 'Working_Data/'

/Users/vitthal/miniforge3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#  Function Defination --> Create backup for test folder
def Backup():
    print('Creating backup for test folder...')
    # Copy data from test folder inside test folder
    try:
        if not os.path.exists(Org_Dir + 'test/'):
            shutil.copytree(Test_Dir1, Org_Dir + 'test/')
    except Exception as e:
        print('Error while creating backup!')
        print(e)
    #  copy data for working folder
    try:
        if not os.path.exists(Work_Dir + 'test/'):
            shutil.copytree(Test_Dir1, Work_Dir + 'test/')
    except Exception as e:
        print('Error while creating backup!')
        print(e)
    # remove test folder
    if os.path.exists(Test_Dir1):
        shutil.rmtree(Test_Dir1)

In [3]:
# Function Defination --> Reconstruct folder
def Reconstruct():
    print('Reconstructing test folder...')
    try:
        if not os.path.exists(Test_Dir1):
            shutil.copytree(Org_Dir + 'test/', Test_Dir1)
    except Exception as e:
        print('Error while reconstructing test folder!')
        print(e)
    # remove backup folder
    if os.path.exists(Org_Dir + 'test/'):
        shutil.rmtree(Org_Dir + 'test/')
    if os.path.exists(Work_Dir + 'test/'):
        shutil.rmtree(Work_Dir + 'test/')

In [4]:
# Function Defination --> Create modality folders for independent cohort
def create_modality_folders():
    print('Creating Modality Folders')
    test_folder = os.listdir(Test_Dir)
    if '.DS_Store' in test_folder:
        test_folder.remove('.DS_Store')
        print('Removed .DS_Store from test folder')
    for pos_neg in tqdm(test_folder):
        patient_folders = os.listdir(os.path.join(Test_Dir, pos_neg))
        # print('Patient Folders: {}'.format(patient_folders))
        if '.DS_Store' in patient_folders:
            patient_folders.remove('.DS_Store')
            print('Removed .DS_Store from patient folder')
        for patient in patient_folders:
            for modality in modalities:
                print('Patient: ',patient)
                modality_folder_path = os.path.join(Test_Dir, pos_neg, modality)
                modality_patient_folder_path = os.path.join(modality_folder_path, patient)
                # print(modality_patient_folder_path)
                if not os.path.exists(modality_folder_path):
                    # print('Creating folder: {}'.format(modality_folder_path))
                    os.makedirs(modality_folder_path)
                if not os.path.exists(modality_patient_folder_path):
                    # print('Creating folder: {}'.format(modality_patient_folder_path))
                    os.makedirs(modality_patient_folder_path)

                modality_file_path = os.path.join(Test_Dir, pos_neg, patient, '{}_{}.nii.gz'.format(patient, modality))
                seg_file_path = os.path.join(Test_Dir, pos_neg, patient, '{}_seg.nii.gz'.format(patient))

                if os.path.exists(modality_file_path) and os.path.exists(seg_file_path):
                   shutil.copy(modality_file_path, modality_patient_folder_path)
                   shutil.copy(seg_file_path, modality_patient_folder_path)
                else:
                    print('File not found: {}'.format(modality_file_path))
                    print('File not found: {}'.format(seg_file_path))
            # delete the patient folder
            shutil.rmtree(os.path.join(Test_Dir, pos_neg, patient))

In [5]:
# Function Defination --> Generating images
def Generate_images():
    try:
        testdir = os.listdir(Test_Dir)
        if '.DS_Store' in testdir:
            testdir.remove('.DS_Store')
            print('Removed .DS_Store from test folder')
        for modality in testdir:
            Modality_path = os.path.join(Test_Dir, modality + '/')
            modalitypath = os.listdir(Modality_path)
            if '.DS_Store' in modalitypath:
                modalitypath.remove('.DS_Store')
                print('Removed .DS_Store from modality folder')
            for patient in modalitypath:
                Patient_path = os.path.join(Modality_path, patient + '/')
                patientpath = os.listdir(Patient_path)
                if '.DS_Store' in patientpath:
                    patientpath.remove('.DS_Store')
                    print('Removed .DS_Store from patient folder')
                for file in patientpath:
                    File_path = os.path.join(Patient_path, file + '/')
                    filepath = os.listdir(File_path)
                    if '.DS_Store' in filepath:
                        filepath.remove('.DS_Store')
                        print('Removed .DS_Store from file folder')
                    os.chdir(File_path)
                    pat = file.split('_')[0]+'_'+file.split('_')[1]

                    list_of_patients = []

                    if pat not in list_of_patients:
                        list_of_patients.append(pat)
                        mod = nib.load('{}_{}.nii.gz'.format(pat, patient))
                        mod_data = mod.get_fdata()
                        seg_mask = nib.load('{}_seg.nii.gz'.format(pat))
                        seg_mask_data = seg_mask.get_fdata()

                        # Extracting layers from mask that have non zero values
                        z = np.any(seg_mask_data, axis=(0,1))
                        zmin, zmax = np.where(z)[0][[0, -1]]  #  zmin & zmax are the first and last layer number non zero values in the z axis

                        # Creating a new mask to remove segmentation
                        d = seg_mask_data
                        for layer in range(zmin, zmax+1):
                             nonzero = np.nonzero(d[:,:,layer])
                             r = nonzero[0]
                             c = nonzero[1]
                             if (r.size == 0) or (c.size == 0):
                                continue
                             rmin = np.min(r)
                             rmax = np.max(r)
                             cmin = np.min(c)
                             cmax = np.max(c)
                             d[rmin:rmax+1, cmin:cmax+1, layer] = 1 #Replacing tumor region values by 1

                        #  Multiply modality data with the new segmentation mask
                        tumor = np.multiply(mod_data, d)

                        # Removing Zero valued layers
                        tumor_layers = tumor[:,:,~(tumor==0).all((0,1))]

                        # Converting to png files
                        Cropped_list = []
                        for lay in range(0, tumor_layers.shape[2]):
                            coords = np.argwhere(tumor_layers[:,:,lay])
                            x_min, y_min = coords.min(axis=0)
                            x_max, y_max = coords.max(axis=0)
                            cropped = tumor_layers[x_min:x_max+1, y_min:y_max+1, lay]
                            cropped *= (255.0/cropped.max()) # Normalizing the values
                            Cropped_list.append(cropped)
                            
                        frame =0
                        for item in Cropped_list:
                            if ((item.shape[0]*item.shape[1])>= 300):
                                frame = frame + 1
                                im = Image.fromarray(item)
                                im = im.convert('L')
                                im.save('{}_{}_{}.png'.format(pat, patient, frame))
                                im.close()
                        
                        # Deleting the nifti files
                        niipath = os.listdir(File_path)
                        if '.DS_Store' in niipath:
                            niipath.remove('.DS_Store')
                            print('Removed .DS_Store from nifti folder')
                        for nii in niipath:
                            try:
                                if nii.startswith(pat) and nii.endswith('.gz'):
                                    os.remove(nii)
                            except Exception as e:
                                print('Error in deleting nifti files')
                                print(e)
    except Exception as e:
        print('Error in Generate_images()')
        print(e)

In [6]:
# Function Defination --> Interpolation

def Interpolation():
    try:
        testdir = os.listdir(Test_Dir)
        if '.DS_Store' in testdir:
            testdir.remove('.DS_Store')
            print('Removed .DS_Store from test folder')
        for modality in testdir:
            Modality_path = os.path.join(Test_Dir, modality + '/')
            modalitypath = os.listdir(Modality_path)
            if '.DS_Store' in modalitypath:
                modalitypath.remove('.DS_Store')
                print('Removed .DS_Store from modality folder')
            for patient in modalitypath:
                Patient_path = os.path.join(Modality_path, patient + '/')
                patientpath = os.listdir(Patient_path)
                if '.DS_Store' in patientpath:
                    patientpath.remove('.DS_Store')
                    print('Removed .DS_Store from patient folder')
                for file in patientpath:
                    File_path = os.path.join(Patient_path, file + '/')
                    filepath = os.listdir(File_path)
                    if '.DS_Store' in filepath:
                        filepath.remove('.DS_Store')
                        print('Removed .DS_Store from file folder')
                    os.chdir(File_path)

                    
                    pngpath = os.listdir(File_path)
                    if '.DS_Store' in pngpath:
                        pngpath.remove('.DS_Store')
                    for png in pngpath:
                        try:
                            if png.endswith('.png'):
                                image = Image.open(png)
                                image = image.resize(inter_dim, Image.ANTIALIAS)
                                png1 = 'inter_' + png
                                image.save(png1)
                                image.close()
                                os.remove(png)
                        except Exception as e:
                            print('Error in Interpolation() - pngpath')
                            print(e)
    except Exception as e:
        print('Error in Interpolation()')
        print(e)

In [7]:
# Function Defination --> Creating Patches

def Creating_patches(block_h, block_w, stride):
    try:
        testdir = os.listdir(Test_Dir)
        if '.DS_Store' in testdir:
            testdir.remove('.DS_Store')
            print('Removed .DS_Store from test folder')
        for modality in testdir:
            Modality_path = os.path.join(Test_Dir, modality + '/')
            modalitypath = os.listdir(Modality_path)
            if '.DS_Store' in modalitypath:
                modalitypath.remove('.DS_Store')
                print('Removed .DS_Store from modality folder')
            for patient in modalitypath:
                Patient_path = os.path.join(Modality_path, patient + '/')
                patientpath = os.listdir(Patient_path)
                if '.DS_Store' in patientpath:
                    patientpath.remove('.DS_Store')
                    print('Removed .DS_Store from patient folder')
                for file in patientpath:
                    File_path = os.path.join(Patient_path, file + '/')
                    filepath = os.listdir(File_path)
                    if '.DS_Store' in filepath:
                        filepath.remove('.DS_Store')
                        print('Removed .DS_Store from file folder')
                    
                    # print(File_path)
                    os.chdir(File_path)
                    for png in tqdm(glob.glob('*.png')):
                        img = Image.open(png)
                        img_w, img_h = img.size

                        File_Name, extentions = os.path.splitext(png)

                        Save_path = Modality_path
                        # print('Save',Save_path)

                        frame_num= 0
                        count_row = 0

                        for row in range(0,img_h,stride):
                            if (img_h-row >= block_h):
                                count_row += 1
                                count_col = 0

                                for col in range(0, img_w, stride):
                                    if (img_h - col >= block_w):
                                        count_col += 1
                                        frame_num += 1

                                        box = (col, row, col +
                                               block_w, row+block_h)
                                        a = img.crop(box)
                                        a.save(
                                            Save_path + File_Name + '_row_' + str(count_row) + '_col_' + str(count_col) + '.png')

                        img.close()
                        os.remove(png)

        # remove patient folders
        for modality in testdir:
            Modality_path = os.path.join(Test_Dir, modality + '/')
            modalitypath = os.listdir(Modality_path)
            if '.DS_Store' in modalitypath:
                modalitypath.remove('.DS_Store')
                print('Removed .DS_Store from modality folder')
            for patient in modalitypath:
                Patient_path = os.path.join(Modality_path, patient + '/')
                shutil.rmtree(Patient_path)

    except Exception as e:
        print('Error in Creating_patches()')
        print(e)

In [8]:
# Function Defination --> Read images

def read_img():
    print('Reading Images')
    class_num = 0
    data = []
    testdir = os.listdir(Test_Dir)
    if '.DS_Store' in testdir:
        testdir.remove('.DS_Store')
        print('Removed .DS_Store from test folder')
    for pool in testdir:
        pool_dir = Test_Dir + pool + '/'
        pool_dir_list = os.listdir(pool_dir)
        if '.DS_Store' in pool_dir_list:
            pool_dir_list.remove('.DS_Store')
            print('Removed .DS_Store from pool folder')
        for img in tqdm(pool_dir_list):
            try:
                img_array = cv2.imread(os.path.join(pool_dir, img), cv2.IMREAD_GRAYSCALE)
                data.append([img_array, class_num])
            except Exception as e:
                print('Error in read_img()')
                print(e)
        class_num = 1
    return data
        

In [9]:
# Function Defination --> Initialize features and labels

def Initializing_feature_labels(data):
    print('Initializing Features & Labels')
    X = []
    Y = []

    for features, label in data:
        X.append(features)
        Y.append(label)
    print('List Size: ', len(X), len(Y))
    return X, Y


In [10]:
# Function Defination --> Converting() Reshape the list to numpy array

def Converting(block_h, block_w, X, Y):
    print('Converting to Array')
    global x, y

    # # -1 is added to solve dimension mismatch while converting list to an array.
    # if len(X) % (block_h * block_w) != 0:
    #     print('Array Size without Reshape: ', len(X), len(Y))
    # n_samples = len(X) // (block_h * block_w)
    # n_missing = (n_samples + 1) * (block_h * block_w) - len(X)
    # X_padded = np.pad(X, [(0, n_missing)], mode='constant')
    # x = np.array(X_padded).reshape((n_samples + 1, block_h, block_w, 1))

    # n_samples = len(X) // (block_h * block_w)
    # x = np.array(X).reshape((n_samples, block_h, block_w, 1))
    x = np.array(X).reshape((-1, block_h, block_w, 1))
    y = np.array(Y)

    print('Array Size with Reshape: ', len(X), len(y))
    # print('Array Shape with Reshape: ', X.shape, y.shape)

# Function Definition --> Reshape the list to numpy array


# def Converting(block_h, block_w, X, Y):
#     print('Converting to Array')
#     global x, y

#     # Check if number of elements is divisible by block size
#     n_elem = len(X)
#     prod = block_h * block_w
#     if n_elem % prod != 0:
#         n_elem_trunc = n_elem // prod * prod
#         X = X[:n_elem_trunc]
#         Y = Y[:n_elem_trunc]
#         print(
#             f"Truncating arrays to {n_elem_trunc} elements to ensure divisibility by block size")

    # # Reshape arrays
    # x = np.array(X).reshape((-1, block_h, block_w, 1))
    # y = np.array(Y)

    # print('Array Size with Reshape: ', len(X), len(y))
    # print('Array Shape with Reshape: ', x.shape, y.shape)


In [11]:
# Function Defination --> Creating Pickle files

def create_pickle_files():
    # list for storing preprocessing data
    data = []

    # Read Images
    data = read_img()

    # Print size of data
    print('Size of data: ', len(data))

    # Initializing all features & labels of the processed image in the list X & Y
    X = []
    Y = []

    # Initializing the features and labels
    X,Y = Initializing_feature_labels(data)

    # Converting the list into numpy array
    Converting(block_h, block_w, X, Y)

    # Storing the numpy array in pickle file
    pickle_out = open(Test_Dir + "X_test.pickle", "wb")
    pickle.dump(x, pickle_out)
    pickle_out.close()

    pickle_out = open(Test_Dir + "y_test.pickle", "wb")
    pickle.dump(y, pickle_out)
    pickle_out.close()

In [12]:
# Function defination --> Create Testing files

def Test_file_creation():

    # # Create backup folder
    # print('Creating Backup Folder')
    # Backup()
    
    # # Create modality folders
    # print('Creating Modality Folders')
    # create_modality_folders()

    # # Generate Images 
    # print('Generating Images')
    # Generate_images()

    # # Interpolation of images
    # print('Interpolating Images')
    # Interpolation()

    # # Create patches
    # print('Creating Patches')
    # Creating_patches(block_h, block_w, stride)

    # Create pickle files
    print('Creating Pickle Files')  
    create_pickle_files()

In [13]:
# Function Defination --> Load pickle files

def Load_Data():
    pickle_in = open(Test_Dir + "X_test.pickle", "rb")
    X = pickle.load(pickle_in) # Features

    pickle_in = open(Test_Dir + "y_test.pickle", "rb")
    y = pickle.load(pickle_in) # Labels

    return X, y

In [14]:
# Function Defination --> Load Model

def Load_Model():
    model = tf.keras.models.load_model(path + 'Outputs/' + load_model )
    return model

In [15]:
# Function Defination --> PLotting AUC-ROC Curve

def Plotting_AUC_ROC_Curve(X,y,model):
    print('Plotting AUC-ROC Curve')
    probs = model.predict(X)
    fpr, tpr, thresholds = roc_curve(y, probs[:, 1])
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('auc_roc_curve.png')
    plt.show()


In [16]:
# Main cell to execute the functions
# # Reconstruction of folders
# Reconstruct()

# Test file creation for computation on independant cohort
# Test_file_creation()

# Loading data
X, y = Load_Data()

# Print lenght of data
print('Length of data: ', len(X), len(y))

results = []

# Load model
model = Load_Model()

# Plotting the AUC-ROC curve
Plotting_AUC_ROC_Curve(X,y,model)


Length of data:  342000 342000
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-02 16:22:54.184952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 16:22:54.185068: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Plotting AUC-ROC Curve
   20/10688 [..............................] - ETA: 29s  

2023-05-02 16:22:54.568121: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-02 16:22:54.627952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10688/10688 [==============================] - 30s 3ms/step


: 

: 